# Character Recurrent Neural Network
- Mimicing Shakespeare's writing style
- Naive RNN

![alt text](./SimpleRNN.png)

## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import unidecode
import string
import random
import re
import time, math

## 2) Hyperparameter

In [3]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data
### 1) Prepare characters

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [5]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [6]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

es blush'd at him and men of heart
Look'd wondering each at other.

CORIOLANUS:
Hear'st thou, Mars?

AUFIDIUS:
Name not the god, thou boy of tears!

CORIOLANUS:
Ha!

AUFIDIUS:
No more.

CORIOLANUS:
Mea


### 2) Character to tensor

In [7]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor).cuda()

print(char_tensor('ABCdef'))

Variable containing:
 36
 37
 38
 13
 14
 15
[torch.cuda.LongTensor of size 6 (GPU 0)]



### 3) Chunk into input & label

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer

![alt text](./embed_rnn_decode.png)

### 1) Model

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, hidden_size)).cuda()
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2).cuda()

In [10]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

Variable containing:
 36
[torch.cuda.LongTensor of size 1 (GPU 0)]

torch.Size([2, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [12]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        #_,top_i = torch.max(output_dist,dim=0)
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [13]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 Variable containing:
 4.6440
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

b>y9~m
4dkthOF/IzHkD6e$2r'Agr\<kU"@gT$LA5e
onrc(ykU~YN_ZP<u+;B}:yL4}P#[+g1$=if<(=oU(8	&\#_w\8R#i"z3j<u=7*hN}/:R11J/[_^`TQ'V}



 Variable containing:
 2.3813
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bull mf hithaflfong, Gove'd ars mathord thip maistpere aouy chiy arrerend me mards in lourhe herle thather
Woveun picif, hutd machy helor warsid the the lour me bis a lutn
Woge laing, ar harendt my pot



 Variable containing:
 2.0971
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

by tage the the he.

Fillir
The as in, st!

I lonle hater, youn ve'd not sder.
Hes, and ore mis af heress,
Hat the fan? nowhis he axest wid ther stenuf thesfr sirfed that whak bend tile; sian is mertin



 Variable containing:
 2.1463
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

breCpmy uf that that the one cilt, of the geees theer here the on he sfre'lt of the lites hese,
IW
UCETTENIO:
This nod sall hereech and endon blound coud'd

To whill my uppold she dryster: shall dishold doth the daughter is seep to well teet remise the set, that courtuen, they stay whead 



 Variable containing:
 1.7036
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bounts id any!

KING RICHARD:
I him agair;
Rold the quile Clayy,
Even to my lord of alles be liffiance, I will the gents laster, freethour is not fee herefore the so he death, merrof besto, lo,
Of an o



 Variable containing:
 1.6884
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

broint, and with to the ploos prings but his so,
Unlike reto the licks gad
again him senfoustrous house,
And let is this repord it lord,
My lord-no many those puses is by the were up all of the pass,
W



 Variable containing:
 1.7589
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

brant make up me virst my belieitory vire it Clantlice this to my draces.

MERCUTIO:
Why say to her be let here is death.

AUTIO:
A says and yound and painty:
But be have not to my sporn'd me of your s



 Variable containing:

Now; if 



 Variable containing:
 1.6906
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

but do name for surman disconmess pray:
For not not they she death: Now of me the she lasters any much should ground doth and one the may all will all by that my good many my fair andrether with ever b



 Variable containing:
 1.4971
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

but warrerer made not thou birnest in tell,
With lient, that in sir.

Second
Till his swiffold!
she wrot his worl would be may not by that I much contenty ege that nor thee to
All breathe in
hight,
The



 Variable containing:
 1.5444
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bad, this upon wide:
Ye' my eye, the Longs
That seep than that bless any sir'd so faces the harber'd, were them loves death, and his food the sword, he fire all truster the soul.

KING RICHARD III:
Sho



 Variable containing:
 2.1341
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

both voom, here aftencly did on I have be all that no him dear hear s

To fathel sake you not my will rough a perullom:
Arcaudt the orged then I have she ready
As scastt
Fear presorvence, and thou were that your papest.

CAPULEL:
Hare armber
That in a



 Variable containing:
 1.7663
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

beep.

PETRUCHIO:
Yet for these this so who shoring the queent:
What prison? With the proft,
Provinention thee be heart,
Thou the will good me, do the palling and your forgeth of this haw-fore one hors



 Variable containing:
 1.6926
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

blaot hear and the well adon: I'll are slace the stressies! hell.

ARGALO:
There wholemater poin!'
Thought your groad? I terrandines, and not gence so he gown of death!
I'll from of fill my shall again



 Variable containing:
 1.5467
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

body the are we are impitided, where we have comes you sir, the love attizerselnes, the aid thee of you the sided their the mertriolany speak to her here, that must down.

PETRUC

What, in yet thou have pray him, jedancour'd of I mear grace.

KING RICHARD III:
Then go the pasish I say, the dear hears thee she his my reasons, I sporious a have 



 Variable containing:
 1.6347
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

berice to there the queen the pit a thou befend?

CAPULET:
Girful an are her her to hiest to the charce of a may he why day their the most.

LORD BENRES:
A licked.

MERCUTIO:
The bospited be, are not h



 Variable containing:
 1.6087
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

broy what the sweet my a pate; to have should in our child thought the fater I call'd an one way, that your
give the vo!
I present:
I be the delive, sir, fie for good not, so tears to care the sweet be



 Variable containing:
 1.4535
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

best then thou have me, spervingst as everess to-morrowery is gaint too comprift time, sirs tape, by my dead.

PRINCE:
But.

PROSPERO:
No partial trument not our dead
track the hard's newgnets 

How and pause me and time to be Clarencious
To paped dayss.
Thee love, on of to made her a man, no we are of call, sir, I was l



 Variable containing:
 1.5441
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bray's them make then, gentlemony the other, this is that good dost to do done, I the thiedy in him tears then windess
though tales;
Has that the assure that let almoy!

Corishard this sea titters to M



 Variable containing:
 1.5988
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bereponour treats,
Unlest the repart
Since thou deside denitial one see upons to shall savener see they me see, sir, and hast some a faets a Cuilstard, and not, and the now'd me his lord stand of buck?



 Variable containing:
 1.4286
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bressience; sir:
And was resone, hangs againss may minisures you wasider with my use, this heaven that caused the resight, than or?

HENRY HIZEL:
And stretch'd shame to from you have camal, Let me be h



 Variable containing:
 1.6

Sick their a t



 Variable containing:
 1.5655
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

brack,
Thinking.

LUCENTIO:
Loved as the good to it even my faed I have a gellow us appears of the gurrish humble to my execession as with in a most is thy remool's at Cature the juststions of these ay



 Variable containing:
 1.5936
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bether my brace.

PETRUCHIO:
This other with the hard in seek word and faunt fenter siles me my bady;
Now I have some the part them, have I shave the in me awayen upon not of heaven of which he come we



 Variable containing:
 1.4654
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

but your dear at on heaven,
And tell tens,
Be one of the si to my foot pleasure sad to say crial his straise hear to condies.

Pirst made be entertacteron on his but lives of it the world.
Now, in Here



 Variable containing:
 1.9224
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

but in the repsions hence sweat me as is ray,
Not her bring a c

In [14]:
weights = torch.Tensor([0.26, 0.24, 0.25, 0.25])

for i in range(10):
    print(torch.multinomial(weights, 1))


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 3
[torch.LongTensor of size 1]


 0
[torch.LongTensor of size 1]


 0
[torch.LongTensor of size 1]


 0
[torch.LongTensor of size 1]


 3
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]


 2
[torch.LongTensor of size 1]

